# Table of Contents
 
1. Importing Libraries
2. Dataset Description
3. Importing the Dataset

1. Importing Libraries

In [3]:
# data processing
import pandas as pd
import numpy as np 

# visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns


2. Dataset Description

- person_age - Age of the borrower applying for a loan
- person_income - Annual income of the borrower applying for a loan
- person_home_ownership
    - OWN - They own a home
    - RENT - They rent a home
    - MORTGAGE - They have a mortgage on the home they own
    - OTHER - Other categories of home ownership
- person_emp_length - Employment length of borrower in years
- loan_intent - What the borrower intends to use the loan for
- loan_grade 
    - A - The borrower has a high creditworthiness, indicating low risk.
    - B - The borrower is relatively low-risk, but not as creditworthy as Grade A.
    - C - The borrower's creditworthiness is moderate.
    - D - The borrower is considered to have higher risk compared to previous grades.
    - E - The borrower's creditworthiness is lower, indicating a higher risk.
    - F - The borrower poses a significant credit risk.
    - G - The borrower's creditworthiness is the lowest, signifying the highest risk.
- loan_amnt - The loan amount the borrower is requesting
- loan_int_rate - The loan interest rate
- loan_status 	
    - 0 is non-default (The borrower paid every loan payment on time, so no default was indicated)
    - 1 is default (The borrower failed to pay according to the agreed terms, so they defaulted on the loan)
- loan_percent_income - Percentage of income from the loan
- cb_person_default_on_file
    - Y - The borrower has a histroy of defaulting on their loans
    - N - The borrower does not have a history of defaulting on their loans
- cb_preson_cred_hist_length - Crdit history of the borrower

3. Importing the Dataset

In [4]:
# Reading the data and printing a sample

data = pd.read_csv("credit_risk_dataset.csv")

data.sample(5)

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
26995,30,120000,MORTGAGE,6.0,MEDICAL,A,9600,7.66,0,0.08,N,9
2931,24,115154,RENT,8.0,HOMEIMPROVEMENT,C,3000,14.27,0,0.03,N,2
22298,28,54000,MORTGAGE,11.0,HOMEIMPROVEMENT,A,1200,7.51,0,0.02,N,7
23158,29,54000,MORTGAGE,1.0,PERSONAL,G,25000,20.16,1,0.46,Y,8
11925,23,33000,RENT,0.0,DEBTCONSOLIDATION,A,11000,7.66,1,0.33,N,2
